In [1]:
# Importa as bibliotecas
# Validar o mínimo de CPF
# Criar uma home direcionando para o crud e para o gráfico

import os
from dotenv import load_dotenv

import pandas as pd
import psycopg2 as pg
import sqlalchemy
from sqlalchemy import create_engine
import panel as pn
import matplotlib.pyplot as plt

In [2]:
# Carrega as variáveis do arquivo .env

load_dotenv()

True

In [3]:
# Lê as variáveis de ambiente

DB_HOST = os.getenv('DB_HOST')
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASS = os.getenv('DB_PASS')

In [4]:
# Cria conexão com psycopg2 usando as variáveis carregadas

con = pg.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASS)

In [5]:
# Define a string de conexão para o SQLAlchemy, utilizando as variáveis do .env
# Cria o objeto engine do SQLAlchemy que será usado para conectar e executar comandos no banco

cnx = f'postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}'

engine = create_engine(cnx)

In [6]:
# Executa a consulta SQL para buscar todos os 
# registros da tabela 'pessoa' no banco PostgreSQL 
# e carrega o resultado em um DataFrame do pandas


query = "select * from usuario;" 
df = pd.read_sql_query(query, cnx)

df.head()

,id_usuario,nome_completo,email,senha,pais,estado,cidade,rua,num_residencia
0,1,João Silva,joao1@email.com,123,Brasil,SP,São Paulo,Rua A,10.0
1,2,Maria Souza,maria2@email.com,123,Brasil,RJ,Rio de Janeiro,Rua B,20.0
2,3,Carlos Lima,carlos3@email.com,123,Brasil,MG,Belo Horizonte,Rua C,30.0
3,4,Ana Rocha,ana4@email.com,123,Brasil,RS,Porto Alegre,Rua D,40.0
4,5,Pedro Alves,pedro5@email.com,123,Brasil,SC,Florianópolis,Rua E,50.0


In [ ]:
# Inicializa as extensões do Panel necessárias:
# - Tabulator para tabelas interativas
# - Notificações na interface

pn.extension()
pn.extension('tabulator')
pn.extension(notifications=True)

In [ ]:
# Criação da Home sem conectar os cliques ainda
btn_go_user_crud = pn.widgets.Button(name="📋 CRUD Usuário", button_type="primary", width=250)
btn_go_campaign_crud = pn.widgets.Button(name="📋 CRUD Campanha", button_type="primary", width=250)
btn_go_cpoint_crud = pn.widgets.Button(name="📋 CRUD Ponto de Coleta", button_type="primary", width=250)
btn_go_donation_crud = pn.widgets.Button(name="📋 CRUD Doação", button_type="primary", width=250)
btn_go_graficos = pn.widgets.Button(name="📊 Ir para Gráficos", button_type="success", width=250)

In [ ]:
# Container principal da aplicação
main_area = pn.Column()

def create_home_view():
    home = pn.Column(
        pn.pane.Markdown("## 🏠 Tela Inicial"),
        pn.pane.Markdown("Escolha uma opção abaixo:"),
        pn.Spacer(height=20),
        btn_go_user_crud,
        btn_go_campaign_crud,
        btn_go_cpoint_crud,
        btn_go_donation_crud,
        btn_go_graficos,
        align="center",
        sizing_mode="stretch_both",
        margin=(50,50,50,50)
    )
    return home


In [ ]:
# Funções de navegação
def go_to_home():
    main_area.clear()
    main_area.append(create_home_view())

In [ ]:
def create_btn_voltar():
    btn_voltar_home = pn.widgets.Button(name='⬅ Voltar para Home', button_type='default')
    btn_voltar_home.on_click(lambda e: go_to_home())
    return btn_voltar_home

In [ ]:
# Variável auxiliar para consultas sem filtro
flag=''

# Widgets de entrada de dados
def get_user_widgets():
    return {
        "nome_completo": pn.widgets.TextInput(
            name="Nome",
            placeholder="Digite o nome",
            sizing_mode="stretch_width"
        ),
        "email": pn.widgets.TextInput(
            name="Email",
            placeholder="Digite o email",
            sizing_mode="stretch_width"
        ),
        "senha": pn.widgets.PasswordInput(
            name="Senha",
            placeholder="Digite a senha",
            sizing_mode="stretch_width"
        ),
        "pais": pn.widgets.TextInput(
            name="País",
            placeholder="Digite o país",
            sizing_mode="stretch_width"
        ),
        "estado": pn.widgets.TextInput(
            name="Estado",
            placeholder="Digite o estado",
            sizing_mode="stretch_width"
        ),
        "cidade": pn.widgets.TextInput(
            name="Cidade",
            placeholder="Digite a cidade",
            sizing_mode="stretch_width"
        ),
        "rua": pn.widgets.TextInput(
            name="Rua",
            placeholder="Digite a rua",
            sizing_mode="stretch_width"
        ),
        "num_residencia": pn.widgets.IntInput(
            name="Número",
            placeholder="Digite o número",
            sizing_mode="stretch_width"
        )
    }

def get_campaign_widgets():
    return {
        "id": pn.widgets.IntInput(name="ID Campanha", value=0),
        "nome": pn.widgets.TextInput(name="Nome da Campanha"),
        "data_inicio": pn.widgets.DatePicker(name="Data Início"),
        "data_fim": pn.widgets.DatePicker(name="Data Fim"),
        "id_inst": pn.widgets.IntInput(name="ID Instituição Responsável"),
        "meta": pn.widgets.TextInput(name="Meta da Campanha"),
        "regras": pn.widgets.TextInput(name="Regras da Campanha"),
        "qnt_vagas": pn.widgets.IntInput(name="Quantidade de vagas", value=1),
    }

def get_cpoint_widgets():
    return {
        "id": pn.widgets.IntInput(name="ID Ponto", value=0),
        "pais": pn.widgets.TextInput(name="País"),
        "estado": pn.widgets.TextInput(name="Estado"),
        "cidade": pn.widgets.TextInput(name="Cidade"),
        "rua": pn.widgets.TextInput(name="Rua"),
        "num_res": pn.widgets.IntInput(name="Número"),
        "id_camp": pn.widgets.IntInput(name="ID Campanha", value=0)
    }

def get_donation_widgets():
    return {
        "id": pn.widgets.IntInput(name="ID Doação"),
        "data": pn.widgets.DatePicker(name="Data da Doação"),
        "desc": pn.widgets.TextInput(name="Descrição/Item"),
        "id_cont": pn.widgets.IntInput(name="ID Contribuidor"),
        "id_camp": pn.widgets.IntInput(name="ID Campanha"),
        "id_ponto": pn.widgets.IntInput(name="ID Ponto Coleta")
    }

In [ ]:
# Botões de ações CRUD
buttonConsultar = pn.widgets.Button(name='Consultar', button_type='primary')
buttonInserir = pn.widgets.Button(name='Inserir', button_type='success')
buttonExcluir = pn.widgets.Button(name='Excluir', button_type='danger')
buttonAtualizar = pn.widgets.Button(name='Atualizar', button_type='warning')

# Botão para voltar à Home (na tela CRUD)
buttonVoltar = pn.widgets.Button(name='⬅ Voltar para Home', button_type='default')


In [ ]:

def queryAll(table):
    """
    Consulta todos os registros de uma tabela e retorna um Tabulator.
    """
    df = pd.read_sql_query(f"SELECT * FROM {table}", cnx)
    return pn.widgets.Tabulator(df, show_index=False, sizing_mode='stretch_width', height=400)

def executar_sql(query, params=None):
    try:
        cursor = con.cursor()
        cursor.execute(query, params)
        con.commit()
        cursor.close()
        pn.state.notifications.success("Operação realizada com sucesso!")
        return True
    except Exception as e:
        con.rollback()
        pn.state.notifications.error(f"Erro: {str(e)}")
        return False

In [1]:
# CRUD USUARIO
def on_consultar_user(w):
    """Consulta usuario pelo email ou retorna todos."""
    try:
        email = w["email"].value
        query = f"SELECT * FROM usuario WHERE ('{email}'='{flag}' OR email='{email}')"
        df = pd.read_sql_query(query, cnx)
        return pn.widgets.Tabulator(df, show_index=False, sizing_mode='stretch_width', height=400)
    except Exception as e:
        return pn.pane.Alert(f'Erro na consulta: {str(e)}', alert_type='danger')

def on_inserir_user(w):
    """Insere um registro novo."""
    query = "INSERT INTO usuario(nome_completo, email, senha, pais, estado, cidade, rua, num_residencia) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"
    params = (w["nome_completo"].value, w["email"].value, w["senha"].value, w["pais"].value, w["estado"].value, w["cidade"].value, w["rua"].value, w["num_residencia"].value)
    if w['email'].value == '':
        pn.state.notifications.error(f"O campo email é obrigatório!")
    else:
        executar_sql(query, params)
    return queryAll("usuario")

def on_atualizar_user(w):
    """Atualiza pelo email."""
    query ="UPDATE usuario SET nome_completo=%s, senha=%s, pais=%s, estado=%s, cidade=%s, rua=%s, num_residencia=%s WHERE email=%s"
    params = (w["nome_completo"].value, w["senha"].value, w["pais"].value, w["estado"].value, w["cidade"].value, w["rua"].value,
              w["num_residencia"].value, w["email"].value)
    
    if w['email'].value == '':
        pn.state.notifications.error(f"O campo email é obrigatório!")
    else:
        executar_sql(query, params)
    return queryAll("usuario")

def on_excluir_user(w):
    try:
        cursor = con.cursor()
        email = w["email"].value

        if w['email'].value == '':
            pn.state.notifications.error(f"O campo email é obrigatório!")
            return queryAll("usuario")
        
        cursor.execute("SELECT id_usuario FROM usuario WHERE email = %s", (email,))
        result = cursor.fetchone() # retorna uma tupla

        if result:
            id_usuario = result[0]
            cursor.execute("DELETE FROM instituicao WHERE id_instituicao = %s", (id_usuario,))
            cursor.execute("DELETE FROM contribuidor WHERE id_contribuidor = %s", (id_usuario,))

            cursor.execute("DELETE FROM usuario WHERE id_usuario = %s", (id_usuario,))

            con.commit()
            cursor.close()
        
            pn.state.notifications.success("Operação realizada com sucesso!")
        return queryAll("usuario")
    except Exception as e:
        cursor.execute("ROLLBACK")
        cursor.close()
        return pn.pane.Alert(f'Não foi possível excluir: {str(e)}', alert_type='danger')


In [ ]:
# CRUD CAMPANHA
def on_consultar_campaign(w):
    """Consulta pelo nome ou retorna todos."""
    try:
        nome = w["nome"].value
        query = f"SELECT * FROM campanha WHERE ('{nome}'='{flag}' OR nome LIKE '%{nome}%')"
        df = pd.read_sql_query(query, cnx)
        return pn.widgets.Tabulator(df, show_index=False, sizing_mode='stretch_width', height=400)
    except Exception as e:
        return pn.pane.Alert(f'Erro na consulta: {str(e)}', alert_type='danger')

def on_inserir_campaign(w):
    """Insere uma nova campanha."""
    query = """INSERT INTO campanha(nome, meta, regras, data_ini, data_fim, qnt_vagas, id_instituicao) 
               VALUES (%s, %s, %s, %s, %s, %s, %s)"""
    # Note: Regras e qnt_vagas podem ser adicionadas aos widgets se desejar, aqui usei valores padrão/exemplo
    params = (w["nome"].value, w["meta"].value, w["regras"].value, w["data_inicio"].value, w["data_fim"].value, w["qnt_vagas"].value, w["id_inst"].value)
    
    if w['nome'].value == '' or w['id_inst'].value == 0:
        pn.state.notifications.error("Nome e ID da Instituição são obrigatórios!")
    else:
        executar_sql(query, params)
    return queryAll("campanha")

def on_atualizar_campaign(w):
    """Atualiza pelo ID da Campanha."""
    query = """UPDATE campanha SET nome=%s, meta=%s, regras=%s, data_ini=%s, data_fim=%s, qnt_vagas=%s, id_instituicao=%s
            WHERE id_campanha=%s"""
    params = (w["nome"].value, w["meta"].value, w["regras"].value, w["data_inicio"].value, w["data_fim"].value, w["qnt_vagas"].value, w["id_inst"].value, w["id"].value)
    
    if w['id'].value == 0:
        pn.state.notifications.error("Informe o ID da campanha para atualizar!")
    else:
        executar_sql(query, params)
    return queryAll("campanha")

def on_excluir_campaign(w):
    """Exclui campanha pelo ID (Lembrando que doações dependem dela)."""
    try:
        id_camp = w["id"].value
        if id_camp == 0:
            pn.state.notifications.error("Informe o ID para excluir!")
            return queryAll("campanha")
        
        # O banco possui FK, então deletamos dependências primeiro se necessário ou deixamos o erro disparar
        executar_sql("DELETE FROM campanha WHERE id_campanha = %s", (id_camp,))
        return queryAll("campanha")
    except Exception as e:
        return pn.pane.Alert(f'Erro: {str(e)}', alert_type='danger')

In [ ]:
# CRUD PONTO DE COLETA

def on_consultar_cpoint(w):
    """Consulta pelo nome da rua ou cidade, ou retorna todos os pontos."""
    try:
        cidade = w["cidade"].value
        rua = w["rua"].value
        # Filtro flexível: busca por cidade ou rua se preenchidos
        query = f"""
            SELECT * FROM ponto_coleta 
            WHERE ('{cidade}'='{flag}' OR cidade LIKE '%{cidade}%')
            AND ('{rua}'='{flag}' OR rua LIKE '%{rua}%')
        """
        df = pd.read_sql_query(query, cnx)
        return pn.widgets.Tabulator(df, show_index=False, sizing_mode='stretch_width', height=400)
    except Exception as e:
        return pn.pane.Alert(f'Erro na consulta: {str(e)}', alert_type='danger')

def on_inserir_cpoint(w):
    """Insere um novo ponto de coleta vinculado a uma campanha."""
    query = """
        INSERT INTO ponto_coleta(pais, estado, cidade, rua, num_residencia, id_campanha) 
        VALUES (%s, %s, %s, %s, %s, %s)
    """
    params = (
        w["pais"].value, 
        w["estado"].value, 
        w["cidade"].value, 
        w["rua"].value, 
        w["num_res"].value, 
        w["id_camp"].value
    )
    
    # Validação simples de campos obrigatórios
    if w["pais"].value == '' or w["estado"].value == '' or w["cidade"].value == '' or w["rua"].value == '' or w['id_camp'].value == 0:
        pn.state.notifications.error("País, estado, cidade, rua e ID da Campanha são obrigatórios!")
    else:
        executar_sql(query, params)
    return queryAll("ponto_coleta")

def on_atualizar_cpoint(w):
    """Atualiza os dados do ponto de coleta através do ID."""
    query = """
        UPDATE ponto_coleta 
        SET pais=%s, estado=%s, cidade=%s, rua=%s, num_residencia=%s, id_campanha=%s 
        WHERE id_pontocoleta=%s
    """
    params = (
        w["pais"].value, 
        w["estado"].value, 
        w["cidade"].value, 
        w["rua"].value, 
        w["num_res"].value, 
        w["id_camp"].value,
        w["id"].value
    )
    
    if w['id'].value == 0:
        pn.state.notifications.error("Informe o ID do ponto de coleta para atualizar!")
        return queryAll("ponto_coleta")
    else:
        executar_sql(query, params)
    return queryAll("ponto_coleta")

def on_excluir_cpoint(w):
    """Exclui o ponto de coleta pelo ID."""
    try:
        id_ponto = w["id"].value
        if id_ponto == 0:
            pn.state.notifications.error("Informe o ID do ponto para excluir!")
            return queryAll("ponto_coleta")
        
        # O banco possui FK (Horários e Doações podem depender do ponto)
        # Se houver dependências, o erro será exibido pela notificação do executar_sql
        executar_sql("DELETE FROM ponto_coleta WHERE id_pontocoleta = %s", (id_ponto,))
        return queryAll("ponto_coleta")
    except Exception as e:
        return pn.pane.Alert(f'Erro ao excluir: {str(e)}', alert_type='danger')

In [ ]:
# CRUD PARA DOACOES
def on_consultar_donation(w):
    """Consulta todas as doações."""
    return queryAll("doacao")

def on_inserir_donation(w):
    """Insere doação (Data, Descrição, Contribuidor, Campanha, PontoColeta)."""
    query = "INSERT INTO doacao(data, descricao, id_contribuidor, id_campanha, id_pontocoleta) VALUES (%s, %s, %s, %s, %s)"
    params = (w["data"].value, w["desc"].value, w["id_cont"].value, w["id_camp"].value, w["id_ponto"].value)
    
    if w["id_cont"].value == 0 or w["id_camp"].value == 0:
        pn.state.notifications.error("IDs de Contribuidor e Campanha são obrigatórios!")
    else:
        executar_sql(query, params)
    return queryAll("doacao")

def on_atualizar_donation(w):
    """Atualiza descrição da doação pelo ID."""
    query = "UPDATE doacao SET descricao=%s, data=%s, id_contribuidor=%s, id_campanha=%s, id_pontocoleta=%s WHERE id_doacao=%s"
    params = (w["desc"].value, w["data"].value, w["id_cont"].value, w["id_camp"].value, w["id_ponto"].value, w["id"].value)
    if w["id"].value == 0:
        pn.state.notifications.error(f"É necessário um id de doação válido!")
    else: 
        executar_sql(query, params)
    return queryAll("doacao")

def on_excluir_donation(w):
    """Exclui doação pelo ID."""
    if w["id"].value == 0:
        pn.state.notifications.error(f"É necessário um id de doação válido!")
    else:
        executar_sql("DELETE FROM doacao WHERE id_doacao = %s", (w["id"].value,))
    return queryAll("doacao")

In [ ]:
def table_creator(cons, ins, atu, exc, w, id):
    """
    Recebe booleans dos botões e executa a ação correspondente.
    """
    match id:
        case 0:
            if cons: return on_consultar_user(w)
            if ins: return on_inserir_user(w)
            if atu: return on_atualizar_user(w)
            if exc: return on_excluir_user(w)

        case 1:
            # Para Campanha
            if cons: return on_consultar_campaign(w)
            if ins: return on_inserir_campaign(w)
            if atu: return on_atualizar_campaign(w)
            if exc: return on_excluir_campaign(w)

        case 2:
            # Para Ponto de Coleta
            if cons: return on_consultar_cpoint(w)
            if ins: return on_inserir_cpoint(w)
            if atu: return on_atualizar_cpoint(w)
            if exc: return on_excluir_cpoint(w)
        case 3:
            # Para Doações
            if cons: return on_consultar_donation(w)
            if ins: return on_inserir_donation(w)
            if atu: return on_atualizar_donation(w)
            if exc: return on_excluir_donation(w)


In [ ]:
# Cria uma ligação interativa (bind) entre os botões e a função que executa a ação correspondente,
# atualizando a tabela na interface sempre que algum botão for clicado.

def create_interactive_table(widgets, id):
    return pn.bind(
        table_creator,
        buttonConsultar,
        buttonInserir,
        buttonAtualizar,
        buttonExcluir,
        widgets,
        id
    )


In [ ]:
# Tela de CRUD

# Monta o layout da interface com Panel:
# - Coluna esquerda com o título, os campos de entrada e os botões de ação
# - Coluna direita com a tabela interativa que mostra os dados do banco
# O método `.servable()` permite que essa interface seja exibida ao rodar o Panel server

def create_crud_view(title, widgets, id):
    """
    Cria a tela do CRUD.
    """
    crud_layout = pn.Row(
        pn.Column(
            pn.pane.Markdown(f"### 📋 CRUD {title}"),
            *[widgets[k] for k in widgets],
            pn.Row(buttonConsultar, buttonInserir, buttonAtualizar, buttonExcluir),
            pn.Spacer(height=10),
            create_btn_voltar(),
            sizing_mode='stretch_width',
            margin=(20, 20, 20, 20)
        ),
        pn.Column(create_interactive_table(widgets, id), sizing_mode='stretch_both')
    )
    
    return crud_layout


In [ ]:
def go_to_user_crud():
    main_area.clear()
    main_area.append(create_crud_view("Usuário", get_user_widgets(), 0))

# Para Campanha
def go_to_campaign_crud():
    main_area.clear()
    main_area.append(create_crud_view("Campanha", get_campaign_widgets(), 1))

# Para Ponto de Coleta
def go_to_cpoint_crud():
    main_area.clear()
    main_area.append(create_crud_view("Ponto de Coleta", get_cpoint_widgets(), 2))

# Para Doação
def go_to_donation_crud():
    main_area.clear()
    main_area.append(create_crud_view("Doação", get_donation_widgets(), 3))

In [ ]:
# Tela de Gráficos
def grafico_voluntarios_por_campanha():
    query = """
        SELECT c.nome, COUNT(v.id_vaga) AS total_voluntarios
        FROM CAMPANHA c
        LEFT JOIN VAGA v ON c.id_campanha = v.id_campanha
        GROUP BY c.nome
    """
    df = pd.read_sql_query(query, cnx)
    fig, ax = plt.subplots()
    ax.bar(df["nome"], df["total_voluntarios"], color='#2ca02c')
    ax.set_title("Número de Voluntários por Campanha")
    ax.set_ylabel("Quantidade de Voluntários")
    plt.xticks(rotation=45)
    return pn.pane.Matplotlib(fig, tight=True, sizing_mode='stretch_width')

def grafico_doacoes_por_tipo():
    query = """
        SELECT tipo, COUNT(id_doacao) AS total 
        FROM TIPO_DOACAO 
        GROUP BY tipo
    """
    df = pd.read_sql_query(query, cnx)
    fig, ax = plt.subplots()
    ax.pie(df["total"], labels=df["tipo"], autopct='%1.1f%%', startangle=140, colors=plt.cm.Paired.colors)
    ax.set_title("Distribuição por Tipo de Item Doado")
    return pn.pane.Matplotlib(fig, tight=True, sizing_mode='stretch_width')

def grafico_doacoes_por_campanha():
    # Agregação: Conta doações por campanha
    query = """
        SELECT c.nome, COUNT(d.id_doacao) as total 
        FROM campanha c 
        LEFT JOIN doacao d ON c.id_campanha = d.id_campanha 
        GROUP BY c.nome
    """
    df = pd.read_sql_query(query, cnx)
    fig, ax = plt.subplots()
    ax.bar(df["nome"], df["total"], color='skyblue')
    ax.set_title("Quantidade de Doações por Campanha")
    ax.set_ylabel("Total de Doações")
    plt.xticks(rotation=45)
    return pn.pane.Matplotlib(fig, tight=True, sizing_mode='stretch_width')

# Gráfico de usuário por estado
def grafico_qtd_user_por_estado():
    df = pd.read_sql_query("SELECT estado, COUNT(email) AS quantidade FROM usuario GROUP BY estado", cnx)
    fig, ax = plt.subplots()
    ax.bar(df["estado"], df["quantidade"], color=['#1f77b4', '#ff7f0e', '#2ca02c'])
    ax.set_title("Quantidade de Usuários por Estado")
    ax.set_ylabel("Quantidade de Usuários")
    return pn.pane.Matplotlib(fig, tight=True, sizing_mode='stretch_width')

In [ ]:
def create_graficos_view():
    graficos_layout = pn.Column(
        pn.pane.Markdown("### 📊 Análise"),
        grafico_voluntarios_por_campanha,
        grafico_doacoes_por_tipo,
        grafico_qtd_user_por_estado,
        grafico_doacoes_por_campanha,
        create_btn_voltar(),
        sizing_mode='stretch_width',
        margin=(20,20,20,20)
    )
    
    return graficos_layout

In [ ]:
def go_to_graficos():
    main_area.clear()
    main_area.append(create_graficos_view())

In [ ]:
# Conecta os botões da Home às funções de navegação
btn_go_user_crud.on_click(lambda event: go_to_user_crud())
btn_go_campaign_crud.on_click(lambda event: go_to_campaign_crud())
btn_go_cpoint_crud.on_click(lambda event: go_to_cpoint_crud())
btn_go_donation_crud.on_click(lambda event: go_to_donation_crud())
btn_go_graficos.on_click(lambda event: go_to_graficos())

In [ ]:
# Inicializa com a tela Home
go_to_home()

In [ ]:
# App principal, com título e área dinâmica
pn.Column(
    pn.pane.Markdown("# 🏢 Sistema de Funcionários"),
    main_area
).servable()